In [17]:
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder

xmin = 48.863
xmax = 48.869
ymin = 2.327
ymax = 2.335

overpass = Overpass()
query = overpassQueryBuilder(
    bbox = [xmin, ymin, xmax, ymax],
    elementType = 'node',
    selector = '"shop"',
    out = 'body'
)
results = overpass.query(query)

In [18]:
print(results.countNodes())

270


In [19]:
df_dic = {'id': [], 'long':[], 'lat':[]}

for node in results.nodes() :
    df_dic['id'].append(node.id())
    #df_dic['type'].append(node.tags)
    df_dic['long'].append(node.lon())
    df_dic['lat'].append(node.lat())



In [20]:
import pandas as pd

df = pd.DataFrame(df_dic)

In [21]:
df.head()

,id,long,lat
0,247589882,2.330879,48.865340
1,280181365,2.333111,48.868149
2,679466892,2.334043,48.863280
3,682302337,2.334849,48.865303
4,682356619,2.333790,48.866241


In [22]:
df_park = pd.read_csv('stationnement-sur-voie-publique-emprises.csv', sep=';')
df_park.head()

,Régime prioritaire,Régime particulier,Type de stationnement,Arrondissement,Nombre places calculées,Nombre places réelles,Zones résidentielles,Localisation stationnement,Numéro voie,Type voie,...,Numéro séquentiel Tronçon voie,Numéro ilot,Numéro IRIS,Zones ASP,Numéro Section Territoriale de Voirie,Compétence préfecture,1er numéro tronçon voie,Dernier numéro tronçon voie,geo_shape,geo_point_2d
0,2 ROUES,Mixte,Epi,14,8,8,14H,Contre allée,2.0,RUE D',...,750617931,NaN,NaN,14Z3A,STV Sud,NaN,2,2,"{""coordinates"": [[[2.339461066949389, 48.82690...","48.82691741526471, 2.339407122959338"
1,PAYANT MIXTE,Mixte,Longitudinal,15,1,1,15N,Chaussée,NaN,RUE DES,...,750613097,NaN,NaN,15Z2,STV Sud-Ouest,NaN,22,32,"{""coordinates"": [[[2.277740907531597, 48.84372...","48.84374793653563, 2.277707280813084"
2,LIVRAISON,ZL permanente,Longitudinal,3,1,1,3E,Demi lincoln,81.0,RUE,...,1750605824,NaN,NaN,03Z2,STV Centre,NaN,79,89,"{""coordinates"": [[[2.3548619039967265, 48.8639...","48.86396234487453, 2.354904497644908"
3,PAYANT MIXTE,Mixte,Longitudinal,16,9,9,16V,Chaussée,NaN,RUE,...,750612346,NaN,NaN,16Z6,STV Sud-Ouest,NaN,2,10,"{""coordinates"": [[[2.2673016661500354, 48.8457...","48.84591233552975, 2.2674772565237524"
4,PAYANT MIXTE,Mixte,Longitudinal,10,2,2,10E,Chaussée,NaN,RUE DE LA,...,1750602306,NaN,NaN,10Z3,STV Centre,NaN,9,21,"{""coordinates"": [[[2.3652388338961154, 48.8739...","48.87399897155821, 2.3653084082121696"


In [23]:
df_park = df_park[['Régime prioritaire', 'geo_point_2d']]
df_park.head()

,Régime prioritaire,geo_point_2d
0,2 ROUES,"48.82691741526471, 2.339407122959338"
1,PAYANT MIXTE,"48.84374793653563, 2.277707280813084"
2,LIVRAISON,"48.86396234487453, 2.354904497644908"
3,PAYANT MIXTE,"48.84591233552975, 2.2674772565237524"
4,PAYANT MIXTE,"48.87399897155821, 2.3653084082121696"


In [24]:
def not_in_area(pts):
    pts = list(pts.split(','))
    x = float(pts[0])
    y = float(pts[1])
    
    if x > xmin and x < xmax and y > ymin and y < ymax :
        return False
    else :
        return True

In [25]:
indexNames = []

for i in range(len(df_park)) :
    if not_in_area(df_park.loc[i].geo_point_2d) :
        indexNames.append(i)

print(len(df_park))
df_park.drop(indexNames, inplace=True)
print(len(df_park))

64632
329


In [26]:
df_park = df_park.reset_index(drop=True)


indexNames = df_park[ df_park['Régime prioritaire'] != 'LIVRAISON' ].index
print(indexNames)
df_park.drop(indexNames, inplace=True)
len(df_park)

Int64Index([  0,   1,   2,   3,   4,   5,   8,  11,  12,  13,
            ...
            316, 319, 320, 321, 322, 323, 324, 325, 326, 328],
           dtype='int64', length=256)


73

In [27]:
df_park = df_park.reset_index(drop=True)
df_park = df_park.reset_index()

df_park.head()

,index,Régime prioritaire,geo_point_2d
0,0,LIVRAISON,"48.86619677829807, 2.3293569308064885"
1,1,LIVRAISON,"48.86451359579737, 2.331725100913022"
2,2,LIVRAISON,"48.86852543469821, 2.330590984975768"
3,3,LIVRAISON,"48.86657621584841, 2.3349789322618837"
4,4,LIVRAISON,"48.86743198654204, 2.332276688548724"


In [28]:
def dist(x1,x2,y1,y2) :
    x = x1 - x2
    if x < 0 : 
        x = -x
    y = y1 - y2
    if y < 0 : 
        y = -y

    return x + y

def mat_dist(df1, df2):
    mat = []
    for shop in range(len(df1)) :
        shop_dist = []
        for parking in range(len(df2)) :
            x1 = df1.loc[shop].lat
            y1 = df1.loc[shop].long
            
            pts = list(df2.loc[parking].geo_point_2d.split(','))
            x2 = float(pts[0])
            y2 = float(pts[1])
            shop_dist.append(dist(x1,x2,y1,y2))

        mat.append(shop_dist)
    return mat

In [29]:
dist_euclidienne = mat_dist(df, df_park)

In [30]:
from geopy import distance

mat_geo = []
for shop in range(len(df)) :
    shop_dist = []
    for parking in range(len(df_park)) :
        a = (df.loc[shop].lat, df.loc[shop].long)
        
        pts = list(df_park.loc[parking].geo_point_2d.split(','))
        x = float(pts[0])
        y = float(pts[1])
        b = (x, y)
        shop_dist.append(distance.distance(a, b).km)

    mat_geo.append(shop_dist)

In [31]:
from pulp import *

# Paramètres
Dj = 10
N = 73
Ci = 8*60

# Variables
gc1 = dict(df_park['index'])
park = list(gc1.keys())
Xij = LpVariable.dicts('x', park, lowBound=0, cat='float')
Yi = LpVariable.dicts('y', park, lowBound=0, cat='LpBinary')

# Problème 
prblm = LpProblem(name='parking_opti', sense=LpMinimize)

# Contraintes

prblm += (lpSum(Yi[i] for i in len(Yi)) == N)
prblm += (lpSum(Xij[i] for i in len(Xij)) == Dj)
prbml += (lpSum(Xij[i] for i in len(Xij)) <= Ci * lpSum(Yi[i] for i in len(Yi)))
prblm += (Yi[i] for i in len(Yi) == 0 or 1)
prblm += (Xij[i] for i in len(Xij) >= 0)

TypeError: 'int' object is not iterable